# Instalação de bases: CrewAI, API Keys e Modelo LLM


In [1]:
#Instalando dependências da da crew.ai, anthropic, langchain-google-genai
!pip install -q crewai crewai_tools langchain_community #langchain-anthropic google-search-results  #scholarly anthropic langchain-google-genai

In [2]:
#Importanto bibliotecas e configurando chaves de API
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, EXASearchTool
from google.colab import userdata
from langchain_openai import ChatOpenAI
#from langchain_community.utilities.google_scholar import GoogleScholarAPIWrapper
import os
import warnings
#from pydantic import BaseModel, PydanticUserError
#from scholarly import scholarly


In [3]:
warnings.filterwarnings('ignore')

In [4]:
#Configurando chaves API
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["EXA_API_KEY"] = userdata.get('EXA_API_KEY')
os.environ["SERP_API_KEY"] = userdata.get('SERP_API_KEY')

In [5]:
#Configurando as ferramentas da CrewAI
search_tool = SerperDevTool()
exa_search_tool = EXASearchTool()
scrape_tool = ScrapeWebsiteTool()

## Criando os Agentes Inteligentes
Documentação: https://docs.crewai.com/concepts/agents

Agentes:
1.   Agente de Geração de Palavras-chave - keywords_agent
2.   Agente de Busca de Artigos - search_agent
3.   Curador Analítico de Conteúdo Científico - filter_agent
4.   Arquiteto de Síntese Científica - selection_agent



In [6]:
# Agente de Geração de Palavras-chave
keywords_agent = Agent(
    role="Especialista em Mineração Semântica de Palavras-chave",
    goal=(
        "Desenvolver um conjunto sofisticado e estratégico de palavras-chave sobre {tema}, "
        "que capturem a essência conceitual e as nuances interdisciplinares "
        "de um domínio científico específico, maximizando a precisão e "
        "amplitude da prospecção de conhecimento."
    ),
    backstory=(
        "Você é um renomado linguista computacional e cientista de dados "
        "especializado em processamento de linguagem natural e análise "
        "semântica. Com vasta experiência em mapeamento conceitual e "
        "processamento de informações científicas, você desenvolveu uma "
        "metodologia única de extração e categorização de termos que "
        "transcendem os limites disciplinares tradicionais. Sua abordagem "
        "combina análise linguística avançada, inteligência artificial e "
        "compreensão profunda das estruturas de conhecimento científico."
    ),
    verbose=True
)

# Agente de Busca de Artigos
search_agent = Agent(
    role="Estrategista de Prospecção Científica",
    goal=(
        "Realizar busca abrangente e estratégica sobre {tema} em múltiplas bases de dados "
        "científicas, utilizando palavras-chave refinadas para identificar "
        "os artigos mais relevantes, inovadores e impactantes dentro do "
        "escopo de pesquisa definido."
    ),
    backstory=(
        "Você é um cientista de informação com décadas de experiência em "
        "biblioteconomia digital e prospecção científica. Desenvolveu "
        "técnicas avançadas de pesquisa que vão além dos métodos "
        "tradicionais, utilizando análise de redes de citação, impacto "
        "científico e mapeamento de tendências emergentes. Sua expertise "
        "permite navegar complexos repositórios acadêmicos com precisão "
        "cirúrgica e sensibilidade para identificar conhecimento "
        "verdadeiramente transformador."
    ),
    verbose=True,
    allow_delegation=True,
    tools=[search_tool]
)

# Agente de Filtragem de Artigos
filter_agent = Agent(
    role="Curador Analítico de Conteúdo Científico",
    goal=(
        "Aplicar filtros multicamadas altamente sofisticados para selecionar "
        "artigos que representem o estado da arte, considerando rigorosamente "
        "critérios de relevância, metodologia, inovação e potencial de "
        "contribuição para o avanço do conhecimento científico sobre {tema}."
    ),
    backstory=(
        "Você é um pesquisador sênior com experiência interdisciplinar, "
        "tendo atuado em comitês editoriais de revistas científicas "
        "internacionais. Desenvolveu um método singular de avaliação "
        "científica que vai além de métricas tradicionais, considerando "
        "aspectos qualitativos como originalidade metodológica, "
        "potencial de ruptura paradigmática e rigor investigativo. Sua "
        "abordagem combina análise quantitativa de impacto com uma "
        "compreensão qualitativa profunda das nuances científicas."
    ),
    verbose=True,
    allow_delegation=True
)

# Agente de Seleção Final de Artigos
selection_agent = Agent(
    role="Arquiteto de Síntese Científica",
    goal=(
        "Realizar curadoria final altamente discriminativa, selecionando "
        "artigos que não apenas representem excelência científica, mas que "
        "também possibilitem a construção de um panorama integrado e "
        "inovador do conhecimento sobre {tema}."
    ),
    backstory=(
        "Você é um cientista interdisciplinar com uma carreira dedicada "
        "à síntese de conhecimento e identificação de padrões emergentes "
        "na fronteira do conhecimento. Com formação em múltiplas áreas "
        "científicas, desenvolveu a capacidade única de estabelecer "
        "conexões conceituais invisíveis, identificando não apenas o que "
        "já foi descoberto, mas antecipando as próximas grandes questões "
        "de pesquisa. Sua metodologia de seleção vai além da análise "
        "individual de artigos, focando em como o conjunto pode construir "
        "uma narrativa científica coerente e transformadora."
    ),
    verbose=True,
    allow_delegation=True
)

## Criando as tarefas
Documentação: https://docs.crewai.com/concepts/tasks

In [7]:

# Tarefa de Geração de Palavras-chave
gerar_palavras_chave = Task(
    name="Geração de Palavras-chave Científicas",
    description=(
        "Gere um conjunto sofisticado de palavras-chave e termos de pesquisa cientifica"
        "sobre {tema}. Utilize processamento "
        "semântico e mapeamento interdisciplinar para identificar descritores "
        "que capturem nuances conceituais profundas."
    ),
    agent=keywords_agent,
    expected_output=(
        "Lista estruturada de palavras-chave, contendo:\n"
        "- Termos principais\n"
        "- Variações semânticas\n"
        "- Descritores interdisciplinares\n"
        "- Ranking de relevância conceitual"
    ),
    context=[],  # Adicionar contexto conforme necessário
    verbose=True
)

# Tarefa de Varredura de Bases de Dados
buscar_artigos = Task(
    name="Varredura de Bases de Dados Científicas",
    description=(
        "Realize uma busca de artigos científicos em bases de dados científicas e periódicos sobre {tema} "
        "utilizando as palavras-chave da tarefa Geração de Palavras-chave Científicas. Implemente "
        "estratégias avançadas de pesquisa que considerem relevância, "
        "impacto citacional e amplitude conceitual."
        "Utilize artigos que possuem um alto score ou relevancia academica."
    ),
    agent=search_agent,
    expected_output=(
        "Relatório de Prospecção Científica sobre {tema} contendo:\n"
        "- Total de artigos encontrados\n"
        "- Distribuição por base de dados\n"
        "- Análise preliminar de relevância\n"
        "- Mapeamento de tendências iniciais"
    ),
    tools=[exa_search_tool, scrape_tool],
    context=[gerar_palavras_chave],
    verbose=True
)

# Tarefa de Aplicação de Filtros
filtrar_artigos = Task(
    name="Aplicação de Filtros Científicos",
    description=(
        "Aplique filtros sofisticados em todos os artigos "
        "inicialmente selecionados. Utilize critérios multicamadas de "
        "relevância científica, considerando impacto, atualidade, "
        "metodologia e potencial inovador."
    ),
    agent=filter_agent,
    expected_output=(
        "Conjunto de Artigos Filtrados com:\n"
         "- Artigos selecionados\n"
         "- Justificativa de seleção\n"
         "- Pontuação de relevância\n"
         "- Categorização temática"
    ),
    tools=[search_tool, exa_search_tool],
    context=[buscar_artigos],
    verbose=True
)

# Tarefa de Seleção Final de Artigos
selecionar_artigos = Task(
    name="Curadoria Final de Artigos Científicos",
    description=(
       "Realize curadoria final de todos os artigos "
       "filtrados. Execute análise profunda de título, abstract e "
       "potencial de contribuição para o campo de pesquisa."
    ),
    agent=selection_agent,
    expected_output=(
        "Portfolio de Artigos Selecionados contendo:\n"
         "- Artigos definitivamente escolhidos\n"
         "- Resumo executivo de cada seleção\n"
         "- Mapa de conexões conceituais\n"
         "- Identificação de lacunas de conhecimento"
    ),
    tools=[scrape_tool],
    context=[filtrar_artigos],
    verbose=True
)

## Criando a equipe

In [8]:
#grupo de trabalho para post no linkedin
buscar_artigos = Crew(
    agents=[keywords_agent,
            search_agent,
            filter_agent,
            selection_agent],
    tasks=[gerar_palavras_chave,
           buscar_artigos,
           filtrar_artigos,
           selecionar_artigos],
    manager_llm=ChatOpenAI(model="gpt-4o",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Colocando a equipe para trabalhar

In [9]:
tema_pesquisa = "Habilidades importantes para a construcao de Agentes Inteligentes - CrewAI e Langchain"

input = {"tema": tema_pesquisa}

In [10]:
resultado = buscar_artigos.kickoff(inputs=input)

# Agent: Crew Manager
## Task: Gere um conjunto sofisticado de palavras-chave e termos de pesquisa cientificasobre Habilidades importantes para a construcao de Agentes Inteligentes - CrewAI e Langchain. Utilize processamento semântico e mapeamento interdisciplinar para identificar descritores que capturem nuances conceituais profundas.
# Agent: Especialista em Mineração Semântica de Palavras-chave
## Task: Gere um conjunto sofisticado de palavras-chave e termos de pesquisa cientificasobre Habilidades importantes para a construção de Agentes Inteligentes - CrewAI e Langchain. Utilize processamento semântico e mapeamento interdisciplinar para identificar descritores que capturem nuances conceituais profundas.


# Agent: Especialista em Mineração Semântica de Palavras-chave
## Final Answer: 
Para desenvolver um conjunto sofisticado e estratégico de palavras-chave sobre habilidades importantes para a construção de agentes inteligentes utilizando CrewAI e Langchain, é crucial considerar uma

##Formatando a resposta

In [11]:
from IPython.display import Markdown

Markdown(resultado.raw)

Para a criação do Portfólio de Artigos Selecionados focado em Habilidades importantes para a construção de Agentes Inteligentes no contexto de CrewAI e Langchain, a curadoria final envolveu uma análise abrangente dos materiais disponíveis. Optou-se por incluir conteúdo que não apenas se destaque pela excelência prática e atualidade, mas que também contribua para um entendimento integrado e inovador do estado da arte.

1. **Estudos de Caso Seletos**: Incluímos estudos de caso que oferecem insights diretos sobre casos de uso reais de CrewAI e Langchain. Exemplos práticos de implementação proporcionam uma base sólida para entender os desafios e soluções viáveis nesses contextos, facilitando a aplicação em cenários futuros.

2. **Tutoriais Avançados e Vídeos de Especialistas**: O portfólio incorpora tutoriais detalhados que exploram técnicas avançadas no desenvolvimento e aplicação de agentes inteligentes utilizando essas plataformas. Vídeos de especialistas complementam estes recursos, oferecendo um viés mais dinâmico e interativo ao processo de aprendizado.

3. **Discussões em Fóruns e Blogs Técnicos**: Conversas em fóruns de desenvolvedores e blogs técnicos são valiosas por proporcionar uma visão das tendências emergentes e debates correntes em torno de CrewAI e Langchain. A seleção focou em tópicos frequentemente mencionados e analisados de maneira crítica.

4. **Análise de Tendências e Ética**: A inclusão de discussões sobre inovações em automação, padrões de colaboração e questões éticas foram partes essenciais deste portfólio. Isso assegura que o conhecimento selecionado não apenas fomente avanços técnicos, mas também considere impactos sociais e de governança.

5. **Resumos Executivos e Conexões Conceituais**: Cada artigo ou recurso selecionado contém um resumo executivo que sintetiza seu valor e contribuição única. Além disso, foi desenvolvido um mapa de conexões conceituais que interliga os principais temas, tecnologias e tendências, facilitando a navegação pelo portfólio.

6. **Identificação de Lacunas de Conhecimento**: Uma importante conclusão deste trabalho foi a identificação de áreas onde o conhecimento ainda é fragmentado ou em desenvolvimento. Estas lacunas representam oportunidades para futuras pesquisas e inovações, particularmente em questões de interoperabilidade, escalabilidade tecnológica e habilidades de adaptação autônoma dos agentes inteligentes.

Este portfólio não é um fim em si mesmo, mas um ponto de partida para a exploração contínua e aprofundada das capacidades e potências de CrewAI e Langchain dentro do domínio dos agentes inteligentes.